In [1]:
import requests
import pandas as pd
import json
import numpy as np

In [2]:
cols = 'adult, genre_ids, id, original_language, original_title, release_date, vote_average, vote_count'.split(', ')
df = pd.DataFrame(columns=cols)
for page in range(1,11):
    url = f"https://api.themoviedb.org/3/movie/popular?api_key=552dbe069b589a399d500b9bdfcbf6e7&language=en-US&page={page}"
    payload={}
    headers = {}
    response = requests.request("GET", url, headers=headers, data=payload)
    jdata = json.loads(response.text)
    df_temp = pd.DataFrame(jdata['results'])[cols]
    df = df.append(df_temp)


C:\Users\conta\AppData\Local\Temp\ipykernel_11432\3932318407.py:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(df_temp)
C:\Users\conta\AppData\Local\Temp\ipykernel_11432\3932318407.py:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(df_temp)
C:\Users\conta\AppData\Local\Temp\ipykernel_11432\3932318407.py:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(df_temp)
C:\Users\conta\AppData\Local\Temp\ipykernel_11432\3932318407.py:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(df_temp)
C:\Users\conta\AppData\Local\Temp\ipykernel_11432\3932318407.py:10: FutureWarning: The frame

In [3]:
df = df.set_index(df['id'])
df = df.drop_duplicates(subset=['id'])

d:\Anaconda3\lib\site-packages\pandas\core\indexes\base.py:6999: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
  return Index(sequences[0], name=names)


In [4]:
for i in list(df.index):
    if type(df['genre_ids'][i]) == list:
        x = df.loc[i]
        genres = x['genre_ids']
        df.drop(axis = 0, index = i,inplace= True)
        for genre in genres:
            x['genre_ids'] = genre
            df = df.append(x)
    else: pass
df.to_excel('moviestmdb.xlsx')

C:\Users\conta\AppData\Local\Temp\ipykernel_11432\2499428502.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  x['genre_ids'] = genre
C:\Users\conta\AppData\Local\Temp\ipykernel_11432\2499428502.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(x)
C:\Users\conta\AppData\Local\Temp\ipykernel_11432\2499428502.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(x)
C:\Users\conta\AppData\Local\Temp\ipykernel_11432\2499428502.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(x)
C:\Users\conta\Ap

In [5]:
url = "https://api.themoviedb.org/3/genre/movie/list?language=en-US&api_key=552dbe069b589a399d500b9bdfcbf6e7"
response = requests.request("GET", url, headers={}, data={})
jdata = json.loads(response.text)
df_genres = pd.DataFrame(jdata['genres'])
df_genres.to_excel('genres.xlsx')


In [3]:
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium import webdriver
import chromedriver_autoinstaller
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import re
chromedriver_autoinstaller.install()

'C:\\Users\\conta\\AppData\\Roaming\\Python\\Python39\\site-packages\\chromedriver_autoinstaller\\111\\chromedriver.exe'

In [29]:
op = webdriver.ChromeOptions()
op.add_argument('headless')
driver = webdriver.Chrome(options = op)
driver.get('https://www.rottentomatoes.com/')


df_tom = pd.DataFrame(columns = 'tomatoscore, audience_score, lengh, original_title'.split(', '))
df_tom['original_title'] = df['original_title'].unique()
df_tom.set_index('original_title', inplace = True)

for movie in df['original_title'].unique():
    try:
        search_bar =  driver.find_element(By.XPATH  ,'/html/body/div[3]/rt-header/search-algolia/search-algolia-controls/input')
        search_bar.send_keys(movie)
        search_bar.send_keys(Keys.ENTER)

        menu = WebDriverWait(driver, 3).until(EC.presence_of_all_elements_located((By.XPATH, '//*[contains(concat( " ", @class, " " ), concat( " ", "searchNav__filter", " " ))]')))

        for element in menu:
            if(element.get_attribute("data-filter")=='movie'):
                menu_movie = element
                break
        menu_movie.click()
        try:
            menu = WebDriverWait(driver, 1).until(EC.presence_of_element_located((By.PARTIAL_LINK_TEXT,f'{movie}'))).click()
        except: 
            menu = WebDriverWait(driver, 1).until(EC.presence_of_element_located((By.PARTIAL_LINK_TEXT,f'{movie}'.translate({ord(i): None for i in ':-'})))).click()

        tomatoscore =  WebDriverWait(driver, 3).until(EC.presence_of_element_located((By.XPATH,'//*[@id="topSection"]/div[1]/score-board'))).get_attribute('tomatometerscore')
        audience_score =  driver.find_element(By.XPATH,'//*[@id="topSection"]/div[1]/score-board').get_attribute('audiencescore')
        lengh = driver.find_element(By.XPATH,'//*[@id="topSection"]/div[1]/score-board/p')
        lengh = lengh.text.split(', ')[2].replace('h ','.').replace('m','').split('.')
        lengh = int(lengh[0])*60 + int(lengh[1])
        df_tom.loc[movie] = [int(tomatoscore),int(audience_score),lengh]
        print(int(tomatoscore),int(audience_score),lengh)
    except:pass

68 71 95
95 94 102
20 45 107
84 94 161
52 87 130
67 63 100
43 28 78
76 92 192
89 96 116
56 64 80
39 88 125
93 78 102
65 91 117
45 51 100
69 97 126
79 79 107
79 67 93
72 80 126
77 92 123
89 46 103
94 86 132
63 77 125
82 82 155
90 82 132
99 68 93
67 88 91
89 85 122
27 47 91
93 98 148
60 69 83
92 74 117
96 99 131
83 89 92
26 43 105
57 30 105
68 85 128
27 51 111
65 76 112
97 90 117
80 92 139
85 80 138
17 68 93
29 77 147
71 66 97
49 74 112
34 64 93
94 99 135
75 61 140
73 88 112
41 72 126
42 32 90
74 85 126
85 92 149
93 73 99
69 96 122
56 52 189
63 55 107
72 66 105
72 93 106
50 60 86
85 87 176
70 89 87
91 85 105
83 80 130
80 77 116
33 76 106
77 58 93
77 46 93
56 67 75
91 93 99
68 77 93
92 92 140
82 80 161
48 68 105
95 70 100
54 76 126
33 38 97
57 84 86
10 41 127
32 73 77
0 8 88
67 83 105
77 89 101
19 47 105
46 83 142
71 71 124
71 37 77
88 74 157
36 52 121
73 88 106
5 15 89
41 90 116
84 91 139
84 78 153
78 81 138
33 31 87
90 86 141
72 98 110
71 68 149
49 45 88
81 87 96
72 86 165
44 70 112
35 

In [30]:
df_tom

,tomatoscore,audience_score,lengh
original_title,,,
Cocaine Bear,68,71,95
Puss in Boots: The Last Wish,95,94,102
Prizefighter: The Life of Jem Belcher,20,45,107
Black Panther: Wakanda Forever,84,94,161
Sayen,NaN,NaN,NaN
...,...,...,...
Blood,54,57,108
Avengers: Endgame,94,90,181
Faraway,NaN,NaN,NaN


In [ ]:
df_tom.to_excel('rottentomatoes.xlsx')

In [46]:
df_tom = pd.read_excel('rottentomatoes.xlsx')
df_tmdb = pd.read_excel('moviestmdb.xlsx')